In [2]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers, losses
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os.path
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
MildDemented_folder = glob.glob("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset/MildDemented/*.jpg")
MildDemented_imgs = []

In [ ]:
for MildDemented_img in MildDemented_folder:
    MildDemented = cv2.imread(MildDemented_img)
    MildDemented_imgs.append(MildDemented)

In [ ]:
MildDemented_imgs = np.array(MildDemented_imgs)

In [ ]:
ModerateDemented_folder = glob.glob("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset/ModerateDemented/*.jpg")
ModerateDemented_imgs = []

In [ ]:
for ModerateDemented_img in ModerateDemented_folder:
    ModerateDemented = cv2.imread(ModerateDemented_img)
    ModerateDemented_imgs.append(ModerateDemented)

In [ ]:
ModerateDemented_imgs = np.array(ModerateDemented_imgs)

In [ ]:
NonDemented_folder = glob.glob("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset/NonDemented/*.jpg")
NonDemented_imgs = []

In [ ]:
for NonDemented_img in NonDemented_folder:
    NonDemented = cv2.imread(NonDemented_img)
    NonDemented_imgs.append(NonDemented)

In [ ]:
NonDemented_imgs = np.array(NonDemented_imgs)

In [ ]:
VeryMildDemented_folder = glob.glob("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset/VeryMildDemented/*.jpg")
VeryMildDemented_imgs = []

In [ ]:
for VeryMildDemented_img in VeryMildDemented_folder:
    VeryMildDemented = cv2.imread(VeryMildDemented_img)
    VeryMildDemented_imgs.append(VeryMildDemented)

In [ ]:
VeryMildDemented_imgs = np.array(VeryMildDemented_imgs)

In [2]:
data = "/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset"

In [3]:
image_dir = Path(data)
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) 
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)

In [4]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
path = Path("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/AugmentedAlzheimerDataset").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [5]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/Alzheimer Magnetic Reso...,ModerateDemented
1,/content/drive/MyDrive/Alzheimer Magnetic Reso...,ModerateDemented
2,/content/drive/MyDrive/Alzheimer Magnetic Reso...,ModerateDemented
3,/content/drive/MyDrive/Alzheimer Magnetic Reso...,ModerateDemented
4,/content/drive/MyDrive/Alzheimer Magnetic Reso...,ModerateDemented
...,...,...
33979,/content/drive/MyDrive/Alzheimer Magnetic Reso...,MildDemented
33980,/content/drive/MyDrive/Alzheimer Magnetic Reso...,MildDemented
33981,/content/drive/MyDrive/Alzheimer Magnetic Reso...,MildDemented
33982,/content/drive/MyDrive/Alzheimer Magnetic Reso...,MildDemented


In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.2)

In [7]:
train_images = train_datagen.flow_from_directory(
    data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')
validation_images = train_datagen.flow_from_directory(
    data ,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [8]:
inception_model = Sequential()

pretrained_model= tf.keras.applications.InceptionV3(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=4,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

inception_model.add(pretrained_model)

87910968/87910968 [==============================] - 0s 0us/step


In [9]:
inception_model.add(Flatten())
inception_model.add(Dense(512, activation='relu'))
inception_model.add(Dropout(0.2))
inception_model.add(Dense(4, activation='softmax'))

In [10]:
inception_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 22,853,924
Trainable params: 1,051,140
Non-trainable params: 21,802,784
_________________________________________________________________


In [11]:
inception_model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
history = inception_model.fit(train_images,
    steps_per_epoch=len(train_images),
    validation_data=validation_images,
    validation_steps=len(validation_images),
    epochs=10)

Epoch 1/10
850/850 [==============================] - 125s 133ms/step - loss: 0.9668 - accuracy: 0.5611 - val_loss: 0.7932 - val_accuracy: 0.6366
Epoch 2/10
850/850 [==============================] - 107s 126ms/step - loss: 0.7986 - accuracy: 0.6291 - val_loss: 0.7260 - val_accuracy: 0.6652
Epoch 3/10
850/850 [==============================] - 105s 123ms/step - loss: 0.7388 - accuracy: 0.6578 - val_loss: 0.7331 - val_accuracy: 0.6761
Epoch 4/10
850/850 [==============================] - 106s 124ms/step - loss: 0.6995 - accuracy: 0.6833 - val_loss: 0.6768 - val_accuracy: 0.6879
Epoch 5/10
850/850 [==============================] - 105s 124ms/step - loss: 0.6678 - accuracy: 0.6955 - val_loss: 0.6457 - val_accuracy: 0.7054
Epoch 6/10
850/850 [==============================] - 105s 123ms/step - loss: 0.6422 - accuracy: 0.7106 - val_loss: 0.5970 - val_accuracy: 0.7309
Epoch 7/10
850/850 [==============================] - 104s 122ms/step - loss: 0.6146 - accuracy: 0.7208 - val_loss: 0.6015 -

In [12]:
data = "/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/OriginalDataset"

In [13]:
image_dir = Path(data)
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) 
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)

In [14]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
path = Path("/content/drive/MyDrive/Alzheimer Magnetic Resonance Imaging (MRI)/OriginalDataset").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [15]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/Alzheimer Magnetic Reso...,NonDemented
1,/content/drive/MyDrive/Alzheimer Magnetic Reso...,NonDemented
2,/content/drive/MyDrive/Alzheimer Magnetic Reso...,NonDemented
3,/content/drive/MyDrive/Alzheimer Magnetic Reso...,NonDemented
4,/content/drive/MyDrive/Alzheimer Magnetic Reso...,NonDemented
...,...,...
6395,/content/drive/MyDrive/Alzheimer Magnetic Reso...,VeryMildDemented
6396,/content/drive/MyDrive/Alzheimer Magnetic Reso...,VeryMildDemented
6397,/content/drive/MyDrive/Alzheimer Magnetic Reso...,VeryMildDemented
6398,/content/drive/MyDrive/Alzheimer Magnetic Reso...,VeryMildDemented


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
test_images = test_datagen.flow_from_directory(
    data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 6400 images belonging to 4 classes.


In [19]:
test_result = inception_model.evaluate(test_images, verbose=0)[1]
print(f'Metric on test: {test_result}')

Metric on test: 0.7043750286102295
